In [5]:
from azureml.core import Workspace, Environment
from azureml.core.conda_dependencies import CondaDependencies

import plotly.graph_objects as go
import plotly.io as pio
import json

import json
import joblib
import pandas as pd
from azureml.core.model import Model
import os

In [6]:
# Connect to workspace
ws = Workspace.from_config()

# Create environment
env = Environment(name="churn-prediction-env")

# Add dependencies
conda_deps = CondaDependencies()
conda_deps.add_conda_package("scikit-learn")
conda_deps.add_conda_package("pandas")
conda_deps.add_conda_package("numpy")
conda_deps.add_pip_package("joblib")
conda_deps.add_pip_package("azureml-defaults")

# Add visualization libraries
conda_deps.add_conda_package("plotly")
conda_deps.add_conda_package("matplotlib")
conda_deps.add_conda_package("seaborn")

env.python.conda_dependencies = conda_deps

# Register workspace
env.register(workspace=ws)

{
    "assetId": "azureml://locations/polandcentral/workspaces/d5d742c6-a47b-4566-92f6-dd94aaf2a52c/environments/churn-prediction-env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "churn-prediction-env",

In [8]:
# Creating score.py file
score_content = '''
import json
import joblib
import pandas as pd
import numpy as np
from azureml.core.model import Model

complete_pipeline = None

def init():
    global complete_pipeline
    
    try:
        # Load pipeline
        complete_pipeline = joblib.load(Model.get_model_path('churn_complete_model'))
        print("✅ Complete pipeline loaded successfully!")
        
        # Check if all components are available
        if 'model' in complete_pipeline:
            print("✅ Model found in pipeline")
        if 'predict_function' in complete_pipeline:
            print("✅ Predict function found in pipeline")
        if 'preprocessing_info' in complete_pipeline:
            print("✅ Preprocessing info found in pipeline")
            
    except Exception as e:
        print(f"❌ Error loading pipeline: {str(e)}")
        raise

def run(raw_data):
    try:
        # Parse JSON data
        data = json.loads(raw_data)
        print(f"📥 Received data: {data}")
        
        # Convert to DataFrame
        if isinstance(data, dict):
            df = pd.DataFrame([data])
        else:
            df = pd.DataFrame(data)
        
        print(f"📊 DataFrame shape: {df.shape}")
        print(f"📊 DataFrame columns: {df.columns.tolist()}")
        
        # Use predict function from pipeline
        result = complete_pipeline['predict_function'](df)
        
        # Convert numpy arrays to lists for JSON serialization
        response = {
            'predictions': result['predictions'].tolist(),
            'churn_probabilities': result['churn_probabilities'].tolist(),
            'no_churn_probabilities': result['no_churn_probabilities'].tolist()
        }
        
        print(f"📤 Sending response: {response}")
        return json.dumps(response)
        
    except Exception as e:
        error_msg = f"❌ Error during prediction: {str(e)}"
        print(error_msg)
        return json.dumps({"error": error_msg})
'''

# Save score.py
with open('score.py', 'w') as f:
    f.write(score_content)
print("score.py created")

score.py created


In [3]:
print('www')

www


In [7]:
# Register model
model = Model.register(
    workspace=ws,
    model_path="./churn_complete_model.joblib",
    model_name="churn_complete_model",
    description="Complete churn prediction pipeline"
)

Registering model churn_complete_model
